<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/Face_March2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics roboflow opencv-python pillow numpy

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from roboflow import Roboflow
from ultralytics import YOLO
import torch
import shutil
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Connect to Roboflow and download dataset
rf = Roboflow(api_key="2IGtFaicFMGaMwb2mX8A")
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
version = project.version(25)

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Download the dataset in YOLOv8 format
dataset = version.download("yolov8")
print("\nDataset downloaded successfully")


Dataset downloaded successfully


In [8]:
# Set up the dataset properly for YOLOv8
def setup_dataset():
    # Based on the error message, we know where the actual data.yaml is
    yaml_path = "./Face-Detection-25/data.yaml"

    if not os.path.exists(yaml_path):
        print(f"Error: {yaml_path} not found")
        # Search for it
        for root, _, files in os.walk('.'):
            for file in files:
                if file == "data.yaml":
                    yaml_path = os.path.join(root, file)
                    print(f"Found data.yaml at: {yaml_path}")
                    break
            if yaml_path:
                break

    if not os.path.exists(yaml_path):
        print("Could not find data.yaml file")
        return None

    # Load the yaml file to see what's inside
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)

    print("Dataset configuration:")
    for key, value in data_config.items():
        print(f"  {key}: {value}")

    # Fix the paths in the data.yaml file
    base_dir = os.path.dirname(yaml_path)

    # Check if the specified directories exist
    train_dir = os.path.join(base_dir, 'train')
    valid_dir = os.path.join(base_dir, 'valid')
    test_dir = os.path.join(base_dir, 'test')

    print(f"Checking directories:")
    print(f"  Train: {os.path.exists(train_dir)}")
    print(f"  Valid: {os.path.exists(valid_dir)}")
    print(f"  Test: {os.path.exists(test_dir)}")

    # Create necessary directories if they don't exist
    if 'train' in data_config and not os.path.exists(os.path.join(base_dir, 'train', 'images')):
        os.makedirs(os.path.join(base_dir, 'train', 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_dir, 'train', 'labels'), exist_ok=True)

    if 'val' in data_config and not os.path.exists(os.path.join(base_dir, 'valid', 'images')):
        os.makedirs(os.path.join(base_dir, 'valid', 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_dir, 'valid', 'labels'), exist_ok=True)

    # Find and move images if they're not in the expected structure
    for root, _, files in os.walk(base_dir):
        image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        txt_files = [f for f in files if f.endswith('.txt') and f != 'data.yaml']

        if image_files:
            print(f"Found {len(image_files)} images in: {root}")

            # Determine if this is a train, valid, or test directory
            if 'train' in root.lower():
                target_dir = os.path.join(base_dir, 'train', 'images')
                label_dir = os.path.join(base_dir, 'train', 'labels')
            elif 'valid' in root.lower() or 'val' in root.lower():
                target_dir = os.path.join(base_dir, 'valid', 'images')
                label_dir = os.path.join(base_dir, 'valid', 'labels')
            elif 'test' in root.lower():
                target_dir = os.path.join(base_dir, 'test', 'images')
                label_dir = os.path.join(base_dir, 'test', 'labels')
            else:
                # If we can't determine, use 'train' as default
                target_dir = os.path.join(base_dir, 'train', 'images')
                label_dir = os.path.join(base_dir, 'train', 'labels')

            # Create directories if they don't exist
            os.makedirs(target_dir, exist_ok=True)
            os.makedirs(label_dir, exist_ok=True)

            # Move or copy images to the correct location
            for img_file in image_files:
                src_path = os.path.join(root, img_file)
                dst_path = os.path.join(target_dir, img_file)
                if not os.path.exists(dst_path):
                    print(f"  Copying {img_file} to {target_dir}")
                    shutil.copy2(src_path, dst_path)

        if txt_files:
            print(f"Found {len(txt_files)} label files in: {root}")

            # Determine if this is a train, valid, or test directory
            if 'train' in root.lower():
                label_dir = os.path.join(base_dir, 'train', 'labels')
            elif 'valid' in root.lower() or 'val' in root.lower():
                label_dir = os.path.join(base_dir, 'valid', 'labels')
            elif 'test' in root.lower():
                label_dir = os.path.join(base_dir, 'test', 'labels')
            else:
                # If we can't determine, use 'train' as default
                label_dir = os.path.join(base_dir, 'train', 'labels')

            # Create directory if it doesn't exist
            os.makedirs(label_dir, exist_ok=True)

            # Move or copy label files to the correct location
            for txt_file in txt_files:
                src_path = os.path.join(root, txt_file)
                dst_path = os.path.join(label_dir, txt_file)
                if not os.path.exists(dst_path) and txt_file != 'data.yaml':
                    print(f"  Copying {txt_file} to {label_dir}")
                    shutil.copy2(src_path, dst_path)

    # Update data.yaml with correct paths
    data_config['path'] = base_dir

    # Make sure train and val paths are correctly set
    if 'train' in data_config:
        data_config['train'] = os.path.join('train', 'images')
    if 'val' in data_config:
        data_config['val'] = os.path.join('valid', 'images')

    # Write updated yaml
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)

    print(f"Updated dataset configuration file at {yaml_path}")
    return yaml_path

In [ ]:
# Train YOLOv8 model
def train_yolo_model(yaml_path, epochs=30, imgsz=640):
    # Initialize YOLOv8 model
    model = YOLO('yolov8n.pt')  # Start with a pre-trained model

    # Train the model
    try:
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=imgsz,
            patience=10,  # Early stopping patience
            batch=16,
            device=0 if torch.cuda.is_available() else 'cpu',
            name="face_detection_model"
        )
        return model
    except Exception as e:
        print(f"Training error: {e}")
        return None